In [2]:
from src.llm_reviewer.llm_api import make_llm_request, LLMAPIFactory
import os
import sys
from project_path import PROJECT_PATH
sys.path.insert(0, PROJECT_PATH)


service_account_path = PROJECT_PATH + "/creds/google__sa.json"
tracking_sheet_id = "1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4"
delivery_sheet_id = "1eUif5I8xhHU8fY0X9v8r2JI9hWPh7Dq_9VXpSIHwww4"


In [19]:
llm_client = LLMAPIFactory().get()
make_llm_request(
    llm_client,
    [{'role': 'system', 'content': "How are you today this find evening?"}],
    'gpt-4-1106-preview',
    temperature= 0.0,
    max_tokens = 4000,
    response_format = None,
    retries = 3,
    seed=42
)

"As an AI, I don't have feelings or experiences, but I'm here and ready to assist you with any questions or tasks you have. How can I help you today?"

In [20]:
from src.llm_reviewer.llm_api import global_usage_manager

global_usage_manager.print_costs()

Total cost for all models: $0.001


In [22]:
global_usage_manager.reset_usage()

In [16]:
global_usage_manager._usage_dict

defaultdict(<function src.llm_reviewer.llm_api.GlobalUsageManager.__init__.<locals>.<lambda>()>,
            {'gpt-4-1106-preview': defaultdict(int,
                         {'completion_tokens': 36,
                          'prompt_tokens': 15,
                          'total_tokens': 51})})

In [11]:
77*0.03/1000+30*0.01/1000

0.00261

In [1]:
from src.llm_reviewer.notebook_parser import predict_role


/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
predict_role([{'role': 'User', 'content': 'how is my favorite GPT doing today?'}, {'role':  '', 'content': 'I am fine thank you'}, {'role':'User', 'content': 'boring as always'}])

Filling out missing header...


('Assistant', None)

In [23]:

from importlib import reload
from src.llm_reviewer import notebook_reviewer
reload(notebook_reviewer)

/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


<module 'src.llm_reviewer.notebook_reviewer' from '/home/saya/chario/upstream_character_tasks/src/llm_reviewer/notebook_reviewer.py'>

In [45]:
from src.llm_reviewer.notebook_reviewer import review_notebook
import nbformat

fn = 'src/llm_reviewer/tests/samples/10T__ml_research.ipynb'
notebook = nbformat.read(fn, as_version=4)

review = review_notebook(
    {'nb_parsed_notebook': notebook, 'file_id': fn},
    max_threads=10,
    progress_counter= None,
    verbose = 2,
)

Reviews done: 0, Reviews left after this one: 19
Reviews done: 0, Reviews left after this one: 18
Reviews done: 0, Reviews left after this one: 17
Reviews done: 0, Reviews left after this one: 16
Reviews done: 0, Reviews left after this one: 15
Reviews done: 0, Reviews left after this one: 14
Reviews done: 0, Reviews left after this one: 13
Reviews done: 0, Reviews left after this one: 12
Reviews done: 0, Reviews left after this one: 11
Reviews done: 0, Reviews left after this one: 10
Review for turn_id=1 by reviewer='code_reviewer' is done. 1 / 20 reviews completed.
Reviews done: 1, Reviews left after this one: 9
Review for turn_id=3 by reviewer='code_reviewer' is done. 2 / 20 reviews completed.
Reviews done: 2, Reviews left after this one: 8
Review for turn_id=2 by reviewer='english_reviewer' is done. 3 / 20 reviews completed.
Reviews done: 3, Reviews left after this one: 7
Review for turn_id=0 by reviewer='code_reviewer' is done. 4 / 20 reviews completed.
Reviews done: 4, Reviews le

In [30]:
global_usage_manager.print_costs()
global_usage_manager._usage_dict

Total cost for all models: $0.333


defaultdict(<function src.llm_reviewer.llm_api.GlobalUsageManager.__init__.<locals>.<lambda>()>,
            {'gpt-4-1106-preview': defaultdict(int,
                         {'completion_tokens': 2555,
                          'prompt_tokens': 25642,
                          'total_tokens': 28197})})

In [46]:
from src.llm_reviewer.turn_reviewer import load_rubric, load_system_prompt


CODE_PROMPT = load_system_prompt('aggregated_reviews_code_judge')
CODE_RUBRICS = load_rubric('code_reviewer')
LANG_PROMPT = load_system_prompt('aggregated_reviews_english_judge')
LANG_RUBRICS = load_rubric('english_reviewer')


from src.llm_reviewer.llm_api import make_llm_request, LLMAPIFactory

def get_judgment(reviewer, feedback):
    llm_client = LLMAPIFactory().get()
    if reviewer == 'code':
        prompt = CODE_PROMPT
        rubrics = CODE_RUBRICS
    elif reviewer == 'lang':
        prompt = LANG_PROMPT
        rubrics = LANG_RUBRICS
    else:
        raise ValueError("Reviewer type not recognized.")
    
    formatted_prompt = prompt.format(FEEDBACK=feedback, GRADING_RUBRICS=rubrics)
    judgment = make_llm_request(
        llm_client,
        [{'role': 'system', 'content': formatted_prompt}],
        'gpt-4-1106-preview',
        temperature= 0.0,
        max_tokens = 4000,
        response_format = {'type': "json_object"},
        retries = 3,
    )
    return judgment

# Example of running the function
#judgment = get_judgment(CODE_PROMPT, gpt_reviews_df.iloc[0]['code_feedback'])
#judgment


In [47]:
from src.llm_reviewer.notebook_reviewer import notebook_reviews_to_df, IssueLevel

df = notebook_reviews_to_df(filter(None, [review]), IssueLevel.MEDIUM)

In [43]:
review

NameError: name 'review' is not defined

In [48]:
from src.llm_reviewer.notebook_reviewer import notebook_reviews_to_df, IssueLevel

df = notebook_reviews_to_df(filter(None, [review]), IssueLevel.MEDIUM)
get_judgment('code', df.iloc[0]['code_feedback'])

{'judgment': 'The code provided across turns generally functions correctly but frequently lacks error handling, efficiency, and best practices, with critical issues in handling specific cases and assumptions about the environment.',
 'score': 3}

In [52]:
print(df.iloc[0]['code_feedback'])

#Turn 1:

## Code(4/5):
**Medium_Issues**
The code provided assumes the user has Python and the necessary permissions to create a virtual environment and install packages. It does not handle potential errors or provide alternative instructions for different setups.


#Turn 2:

## Code(5/5):
None


#Turn 3:

## Code(3/5):
**Medium_Issues**
The code could be refactored to avoid repetition and improve maintainability by creating a function to fit and evaluate models.
Scaling is applied to all models, but not all models benefit from feature scaling, such as tree-based models.


#Turn 4:

## Code(5/5):
None


#Turn 5:

## Code(4/5):
**Medium_Issues**
The code lacks exception handling for potential errors that may arise during data loading or model training. Adding try-except blocks could improve robustness.


#Turn 6:

## Code(3/5):
**Critical_Issues**
The code lacks error handling for the case where the noise level is too high, potentially resulting in negative feature values which may not

In [35]:
global_usage_manager.print_costs()
global_usage_manager._usage_dict

Total cost for all models: $0.345


defaultdict(<function src.llm_reviewer.llm_api.GlobalUsageManager.__init__.<locals>.<lambda>()>,
            {'gpt-4-1106-preview': defaultdict(int,
                         {'completion_tokens': 2594,
                          'prompt_tokens': 26731,
                          'total_tokens': 29325})})

In [36]:
0.345/10

0.034499999999999996

In [42]:
0.345*100

34.5

In [38]:
0.333/10

0.0333

In [44]:
(0.0333*4+0.012)*100

14.520000000000003

In [40]:
 0.012

0.012

In [39]:
global_usage_manager.print_costs()
global_usage_manager._usage_dict

Total cost for all models: $0.345


defaultdict(<function src.llm_reviewer.llm_api.GlobalUsageManager.__init__.<locals>.<lambda>()>,
            {'gpt-4-1106-preview': defaultdict(int,
                         {'completion_tokens': 2594,
                          'prompt_tokens': 26731,
                          'total_tokens': 29325})})

In [ ]:
from src.llm_reviewer.notebook_reviewer import review_notebook
import nbformat

fn = 'src/llm_reviewer/tests/samples/10T__ml_research.ipynb'
notebook = nbformat.read(fn, as_version=4)

review = review_notebook(
    {'nb_parsed_notebook': notebook, 'file_id': fn},
    max_threads=10,
    progress_counter= None,
    verbose = 2,
)

In [ ]:
You are a concise expert in evaluating and refining the code generated by an AI assistant based on a Large Language Model. You only point out things worth mentioning.

Attributes to consider:
- Code Correctness
- Code Efficiency
- Best Practices
- Code Readability
- Code style Consistency

**1. Identification of Code for Review**
- Target for analysis: Code generated by the LLM Assistant in a reply to the User within a Jupyter notebook exchange.
- Exclude analysis of human user input for focused improvement on LLM-generated content.
- Exclude LLM Assistant text content, only review code snippets and code cells. Text is for context and reasoning/explanation only.
- Additional code cells and interactions may precede or follow the one under review, providing context for the analysis.
- Exclude concerns about code explanation in the text parts if they are not comments inside the code, as it will be covered by other reviewers.

**2. Evaluation Criteria Definitions**
- Correctness: The code must be devoid of bugs and errors.
- Efficiency: The code must be optimized for maximum performance.
- Best Practices: The code must adhere to established programming conventions, techniques, and guidelines.
- Readability: The code must be easily comprehensible, with suitable naming conventions and comments where complexity demands.
- Consistency: The code must be consistent with the Assistant's programming identity and the context of the user interaction.

**3. Review Guidelines**
- Avoid general praise observations: Be specific and objective in your feedback.
- Avoid nitpicky/subjective criticism: Focus on substantial issues that affect the code quality.


@GRADING_RUBRIC@

Your analysis response format must be a JSON adhering to the following JSON schema:
```json
@RESPONSE_FORMAT@
```

Number of problems for each criteria might vary from 0 to multiple.

Analyze:
# TURN_DATA_START
@FORMATTED_TURN@
# TURN_DATA_END

Analyze only LLM_REPLY.

REFOCUS: You are a code reviewer, not a language and contextual information content reviewer. Do not mention issues not related to your purpose.
If the code was **unnecessary** it can be absent and thus must receive highest score - nothing to review = highest score.

Reply in the specified JSON format ->


In [53]:
CODE_REVIEWER = """
# IDENTITY

You are an AI named Codia. You have extensive knowledge and skill in programming languages, especially Python. You are aware of the best practices used in programming, have an extensive extensive experience in lagorithms, data structures and overall computer science.

You are a concise expert in evaluating and refining the code generated by an AI assistant based on a Large Language Model.

# GOALS

Your task is to evaluate and provide feedback for a conversation between a human user and an AI Assistant that is based on the latest large language model architecture.
Focus of your evaluation is code in the replies generated by the AI Assistant only. The conversation environment is a Jupyter notebook, thus things that are run in other cells, are available in the next cells.

# RULES

# OUTPUT_FORMAT

Your output must be a JSON object with the following fields:

# critical_severity_issues - list of issues that make the conversation not useful for the human user.
# medium_severity_issues - list of issues that still have a strong influence on the conversation flow and usefulness.
Each issue should point out a specific problem and how to fix it in exec summary fashion. Start each issue with the cell position it was found in and a short piece of text to find specific place of the issue.

# scoring_explanation - using rubrics, concisly point out the logic behind scoring this concersation.

# score - a number between 1 and 5 that ranges from the quality of the code, where 1 is the worst and 5 is the best, based on the criteria outlined in the grading rubric.

### 5 - Excellent
- Well Formatted
- Correct
- Optimal
- Highly readable

### 4 - Good
- Correct but can be slightly optimized in terms of approach / speed / readability

### 3 - Acceptable
- The code is correct but can be significantly improved.
- The code is not readable.

### 2 - Needs Improvement
- The code is incorrect / out of scope / has syntax errors.
- Looks like it’s copied from ChatGPT.

### 1 - Poor
- Incomplete or Code required by context of the interaction but missing.

```json
{
    "critical_severity_issues": list[str],
    "medium_severity_issues": list[str],
    "scoring_explanation": str,
    "score": number
}
```

# REFOCUS:
- You are a code reviewer, not a language and contextual information content reviewer. Do not mention issues not related to your purpose.
- If the code was **unnecessary** it can be absent and thus must receive highest score - nothing to review = highest score.
"""

In [90]:
CODE_REVIEWER = """
# IDENTITY

You are an AI named Codia. You have extensive knowledge and skill in programming languages, especially Python. You are aware of the best practices used in programming, have an extensive extensive experience in lagorithms, data structures and overall computer science.

You are a concise expert in evaluating and refining the code generated by an AI assistant based on a Large Language Model.

# GOALS

Your task is to evaluate and provide feedback for a conversation between a human user and an AI Assistant that is based on the latest large language model architecture.
Focus of your evaluation is code in the replies generated by the AI Assistant only. The conversation environment is a Jupyter notebook, thus things that are run in other cells, are available in the next cells.

# RULES

Attributes to consider:
- Code Correctness
- Code Efficiency
- Best Practices
- Code Readability
- Code style Consistency

**1. Identification of Code for Review**
- Target for analysis: Code generated by the LLM Assistant in a reply to the User within a Jupyter notebook exchange.
- Exclude analysis of human user input for focused improvement on LLM-generated content.
- Exclude LLM Assistant text content, only review code snippets and code cells. Text is for context and reasoning/explanation only.
- Additional code cells and interactions may precede or follow the one under review, providing context for the analysis.
- Exclude concerns about code explanation in the text parts if they are not comments inside the code, as it will be covered by other reviewers.

**2. Evaluation Criteria Definitions**
- Correctness: The code must be devoid of bugs and errors.
- Efficiency: The code must be optimized for maximum performance.
- Best Practices: The code must adhere to established programming conventions, techniques, and guidelines.
- Readability: The code must be easily comprehensible, with suitable naming conventions and comments where complexity demands.
- Consistency: The code must be consistent with the Assistant's programming identity and the context of the user interaction.

**3. Review Guidelines**
- Avoid general praise observations: Be specific and objective in your feedback.
- Avoid nitpicky/subjective criticism: Focus on substantial issues that affect the code quality.

# OUTPUT_FORMAT

Your output must be a JSON object with the following fields:

# critical_severity_issues - list of issues that make the conversation not useful for the human user.
# medium_severity_issues - list of issues that still have a strong influence on the conversation flow and usefulness.
Each issue should point out a specific problem and how to fix it in exec summary fashion. Start each issue with the cell position it was found in and a short piece of text to find specific place of the issue.
Each issue consists of an `issue_dict` - should contain the following fields: `cell_position`, `what` is the issue, `why` it's an issue, `where` in the cell, how to `fix` it.

# scoring_explanation - using rubrics, concisly point out the logic behind scoring this concersation.

# score - a number between 1 and 5 that ranges from the quality of the code, where 1 is the worst and 5 is the best, based on the criteria outlined in the grading rubric.

### 5 - Excellent
- Well Formatted
- Correct
- Optimal
- Highly readable

### 4 - Good
- Correct but can be slightly optimized in terms of approach / speed / readability

### 3 - Acceptable
- The code is correct but can be significantly improved.
- The code is not readable.

### 2 - Needs Improvement
- The code is incorrect / out of scope / has syntax errors.
- Looks like it’s copied from ChatGPT.

### 1 - Poor
- Incomplete or Code required by context of the interaction but missing.

```json
{
    "critical_severity_issues": list[issue_dict],
    "medium_severity_issues": list[issue_dict],
    "scoring_explanation": str,
    "score": number
}
```

# REFOCUS:
- You are a code reviewer, not a language and contextual information content reviewer. Do not mention issues not related to your purpose.
- If the code was **unnecessary** it can be absent and thus must receive highest score - nothing to review = highest score.
"""

In [85]:
from src.llm_reviewer.llm_api import global_usage_manager
global_usage_manager.reset_usage()

In [87]:

def make_review_request(nb):
    llm_client = LLMAPIFactory().get()
    cells = [{'type': cell['cell_type'], 'content': cell['source']} for cell in nb['cells']]
    cells_with_numbers = [{'cell_position': i + 1, **c,} for i, c in enumerate(cells)]
    r = make_llm_request(
        llm_client,
        [{'role': 'system', 'content': CODE_REVIEWER}, {'role': 'user', 'content': f'# CONVERSATION_START\n\n\n{cells_with_numbers}\n\n\n# CONVERSATION_END'}],
        'gpt-4-1106-preview',
        temperature=0.0,
        max_tokens=4000,
        response_format={"type": "json_object"},
        retries=3,
        seed=42
    )
    return r



In [93]:
from IPython.display import display, Markdown

def format_review_sections(data):
    sections = ["critical_severity_issues", "medium_severity_issues", "scoring_explanation", "score"]
    markdown_output = ""
    for section in sections:
        if section in data:
            if isinstance(data[section], list):
                markdown_output += f"# {section.replace('_', ' ').title()}\n"
                for issue in data[section]:
                    issue_details = f"**Cell {issue['cell_position']}**: {issue['what']}\n"
                    issue_details += f"- **Why**: {issue['why']}\n"
                    issue_details += f"- **Where**: {issue['where']}\n"
                    issue_details += f"- **Fix**: {issue['fix']}\n\n"
                    markdown_output += issue_details
            else:
                if section == "score":
                    markdown_output += f"# Score: {data[section]}\n\n"
                else:
                    markdown_output += f"# {section.replace('_', ' ').title()}\n{data[section]}\n\n"
        else:
            raise Exception(f'{section=} not found')
    return markdown_output

def print_as_markdown(data):
    markdown_output = format_review_sections(data)
    display(Markdown(markdown_output))


In [94]:
r = make_review_request(notebook)
print_as_markdown(r)

# Critical Severity Issues
# Medium Severity Issues
**Cell 8**: Inconsistent use of random_state in SVC initialization.
- **Why**: The SVC classifier is initialized with a random_state parameter, which is not necessary since the 'linear' kernel does not use randomness. This could lead to confusion about the behavior of the algorithm.
- **Where**: In the code snippet initializing the SVC classifier.
- **Fix**: Remove the random_state parameter from the SVC initialization: `svm_classifier = SVC(kernel='linear')`.

**Cell 12**: Inconsistent random_state usage in classifiers.
- **Why**: The RandomForestClassifier and GradientBoostingClassifier are initialized without a random_state, which could lead to non-reproducible results.
- **Where**: In the code snippet initializing RandomForestClassifier and GradientBoostingClassifier.
- **Fix**: Add a random_state parameter to the RandomForestClassifier and GradientBoostingClassifier initializations to ensure reproducibility: `RandomForestClassifier(n_estimators=100, random_state=42)` and `GradientBoostingClassifier(random_state=42)`.

**Cell 14**: The function `add_noise` does not ensure that the noisy data stays within valid bounds.
- **Why**: Adding noise to the data could potentially result in negative values for features that should only have non-negative values, which could lead to incorrect model training.
- **Where**: In the `add_noise` function definition.
- **Fix**: After adding noise, clip the values to ensure they stay within the valid range of the original data: `return np.clip(X_train + X_train*noise, a_min=0, a_max=None)`.

**Cell 16**: The grid search in `noise_level_search_on_iris.py` does not save the results for plotting.
- **Why**: The function `grid_search_noise_levels` returns early if any classifier's accuracy drops below 0.5, which means the results for higher noise levels are not collected and cannot be plotted.
- **Where**: In the `grid_search_noise_levels` function.
- **Fix**: Remove the early return statement to ensure all noise levels are tested and results are collected for plotting.

**Cell 20**: Incorrect calculation of bar width in `plot_results_cluster_bar` function.
- **Why**: The bar width is calculated as `0.1 / len(classifier_names)`, which does not take into account the number of noise levels and may result in overlapping bars.
- **Where**: In the `plot_results_cluster_bar` function, when setting the width variable.
- **Fix**: Calculate the bar width based on the number of classifiers and the spacing needed between bars for each noise level: `width = (1 / len(noise_levels)) / len(classifier_names)`.

# Scoring Explanation
The code provided by the Assistant is mostly correct and follows best practices, with no critical issues that would make the conversation not useful for the user. However, there are several medium severity issues related to consistency and best practices that could be improved. These include unnecessary parameters, missing parameters for reproducibility, potential data range issues, incomplete data collection for plotting, and incorrect bar width calculation for the cluster bar chart. Addressing these issues would improve the code's consistency, reproducibility, and the accuracy of the visualizations.

# Score: 4



In [95]:
print_as_markdown(make_review_request(notebook))

# Critical Severity Issues
# Medium Severity Issues
**Cell 8**: Inconsistent use of random_state in SVC initialization.
- **Why**: The random_state parameter is set in SVC but not in other classifiers that also have this parameter. This inconsistency can lead to different results when the code is run multiple times and makes it harder to reproduce the results.
- **Where**: In the initialization of SVC: `svm_classifier = SVC(kernel='linear', random_state=42)`.
- **Fix**: Either remove the `random_state=42` from the SVC initialization or add a `random_state=42` to all other classifiers that support this parameter to ensure consistency.

**Cell 12**: Inconsistent use of random_state in classifiers.
- **Why**: The random_state parameter is omitted in the initialization of some classifiers in the `classifiers_set.py` file. This can lead to non-reproducible results when the classifiers involve randomness.
- **Where**: In the initialization of classifiers like SVC, RandomForestClassifier, and GradientBoostingClassifier.
- **Fix**: Add `random_state=42` to the initialization of all classifiers that support this parameter to ensure reproducibility.

**Cell 14**: The function `add_noise` does not ensure that the noisy data stays within valid bounds.
- **Why**: Adding noise to the data can potentially result in feature values that are outside the valid range for the dataset, which could lead to unrealistic or invalid data points.
- **Where**: In the `add_noise` function definition: `return X_train + X_train*noise`.
- **Fix**: Implement a check to ensure that the noisy data is clipped within the valid range of the dataset's features.

**Cell 16**: The grid search in `noise_level_search_on_iris.py` does not save the results for plotting.
- **Why**: The function `grid_search_noise_levels` prints the results but does not return them, which makes it impossible to use the results for plotting or further analysis.
- **Where**: In the `grid_search_noise_levels` function definition.
- **Fix**: Modify the function to return the results instead of just printing them.

**Cell 20**: Incorrect calculation of bar width in the `plot_results_cluster_bar` function.
- **Why**: The calculation of the bar width is based on the number of classifiers, which could result in overlapping bars if there are many classifiers.
- **Where**: In the `plot_results_cluster_bar` function definition: `width = 0.1 / len(classifier_names)`.
- **Fix**: Adjust the bar width calculation to take into account the number of noise levels and the spacing between the bars to prevent overlap.

# Scoring Explanation
The code provided by the Assistant is correct and functional, with no critical issues that would make the conversation not useful for the human user. However, there are several medium severity issues related to consistency and best practices, such as inconsistent use of random_state and potential issues with data bounds after adding noise. These issues do not prevent the code from running but could affect the reproducibility and realism of the results. The code is also readable and follows good style conventions.

# Score: 4



In [96]:
print_as_markdown(make_review_request(notebook))

# Critical Severity Issues
# Medium Severity Issues
**Cell 8**: Inconsistent use of random_state in SVC initialization.
- **Why**: The random_state parameter is set in SVC but not in other classifiers that also have this parameter, which may lead to inconsistent reproducibility across different classifiers.
- **Where**: In the SVC initialization: SVC(kernel='linear', random_state=42).
- **Fix**: Either remove the random_state parameter from SVC or add it to all other classifiers that support it to maintain consistency.

**Cell 12**: Inconsistent random_state usage across classifiers.
- **Why**: The random_state parameter is omitted in the SVC and GradientBoostingClassifier initializations, which may affect reproducibility.
- **Where**: In the initializations of SVC and GradientBoostingClassifier: SVC(kernel='linear') and GradientBoostingClassifier().
- **Fix**: Add the random_state parameter to SVC and GradientBoostingClassifier initializations to ensure consistent reproducibility.

**Cell 14**: The function add_noise does not ensure that the noise added is exactly 5% of the feature value.
- **Why**: The noise is generated using a uniform distribution between -noise_level and noise_level, which does not guarantee that the noise is exactly 5% of the feature value.
- **Where**: In the add_noise function: noise = np.random.uniform(-noise_level, noise_level, X_train.shape).
- **Fix**: Adjust the noise generation to ensure that it is exactly 5% of the feature value, for example by using np.random.normal with a mean of 0 and a standard deviation that reflects 5% of the feature values.

**Cell 16**: The grid search increments are not 5% as stated.
- **Why**: The code comment suggests that the noise level is split into 5% increments, but the actual implementation uses np.linspace(0, 1, 21) which creates increments of approximately 5% but not exactly.
- **Where**: In the grid_search_noise_levels function: noise_levels = np.linspace(0, 1, 21).
- **Fix**: Correct the comment to reflect the actual implementation or adjust the linspace parameters to create exact 5% increments.

**Cell 20**: The width calculation for the cluster bar chart is incorrect.
- **Why**: The width is calculated as 0.1 / len(classifier_names), which does not take into account the number of noise levels and may result in bars that are too thin or overlap.
- **Where**: In the plot_results_cluster_bar function: width = 0.1 / len(classifier_names).
- **Fix**: Adjust the width calculation to account for the number of noise levels and classifiers to ensure the bars are appropriately sized and spaced.

# Scoring Explanation
The code provided by the Assistant is correct and functional, with no critical issues that would prevent it from being useful to the user. However, there are several medium severity issues related to consistency and accuracy in the implementation details, such as the use of random_state and the noise level increments. These issues do not prevent the code from running but could lead to confusion or incorrect assumptions about the code's behavior. The code is generally well-structured and readable, but these issues prevent it from being rated higher.

# Score: 4



In [97]:
print_as_markdown(make_review_request(notebook))

In [ ]:
print_as_markdown(make_review_request(notebook))

# Critical Severity Issues
# Medium Severity Issues
- Cell 8: The code snippet uses a dictionary with string keys ('algo' and 'scale') to store classifiers and whether they need scaling. It would be more appropriate to use a tuple or a custom class for this purpose to ensure type safety and better readability.
- Cell 14: The function `add_noise` should ensure that the noise added does not cause feature values to become negative, which could be nonsensical for certain datasets. A check or a different noise application strategy could be implemented to prevent this.
- Cell 20: The code uses a print statement to output `all_results`, which is unnecessary for the final plot and should be removed to clean up the code. Additionally, the width calculation for the bars in the cluster bar chart is based on the number of classifiers, which may result in extremely thin bars if there are many classifiers. A fixed width or a minimum width threshold would be more appropriate.
# Scoring Explanation
The code provided by the Assistant is correct and functional, with no critical issues that would prevent it from being useful to the user. However, there are some medium severity issues related to best practices and code readability that could be improved. The use of dictionaries for storing classifiers and their properties could be replaced with tuples or custom classes for better type safety and readability. The noise addition function should ensure that the noise does not lead to nonsensical data. The plotting code could be cleaned up by removing unnecessary print statements and adjusting the bar width calculation for better visualization. These issues do not severely impact the functionality but do affect the overall quality of the code.

# Score: 4



In [ ]:
print_as_markdown(make_review_request(notebook))

# Critical Severity Issues
# Medium Severity Issues
- {'cell_position': 8, 'issue': 'Inconsistent use of random_state in SVC initialization.', 'fix': 'Add `random_state=42` to the SVC initialization for consistency with other classifiers.'}
- {'cell_position': 12, 'issue': 'Missing random_state in SVC initialization.', 'fix': 'Add `random_state=42` to the SVC initialization for consistency with other classifiers.'}
- {'cell_position': 14, 'issue': 'Inefficient code structure in `run_classifiers` function.', 'fix': 'Refactor the `run_classifiers` function to avoid repeated code by using a single loop to handle both scaled and unscaled data.'}
- {'cell_position': 16, 'issue': 'Inefficient grid search implementation.', 'fix': 'Optimize the grid search by stopping the loop once a classifier accuracy falls below 50% instead of continuing to test higher noise levels.'}
- {'cell_position': 20, 'issue': 'Inefficient bar width calculation.', 'fix': 'Change the bar width calculation to be based on the number of noise levels instead of the number of classifiers.'}
# Scoring Explanation
The code provided by the Assistant is correct and functional, with no critical issues that would make the conversation not useful for the human user. However, there are several medium severity issues related to code efficiency and best practices, such as inconsistent use of random_state and inefficient code structures. These issues do not prevent the code from running correctly but could be optimized for better performance and maintainability. The code readability is generally good, and the Assistant follows consistent code style throughout the conversation.

# Score: 4



In [ ]:
print_as_markdown(make_review_request(notebook))

# Critical Severity Issues
# Medium Severity Issues
- {'issue': 'Cell 8: Inconsistent use of random_state', 'fix': 'Ensure that all classifiers that support a random_state parameter are initialized with it for reproducibility.'}
- {'issue': 'Cell 14: Inefficient code structure', 'fix': "Refactor the code to avoid repeating the logic for adding noise and running classifiers, which is already implemented in 'test_with_noise.py'."}
- {'issue': 'Cell 20: Incorrect width calculation for bar chart', 'fix': 'Correct the width calculation for the bars in the cluster bar chart to ensure they are thin and non-overlapping.'}
# Scoring Explanation
The code provided by the Assistant is correct and functional, with no critical issues that would make the conversation not useful for the user. However, there are medium severity issues related to code efficiency and consistency. In Cell 8, the random_state parameter is not consistently used across all classifiers, which could affect reproducibility. In Cell 14, the code structure could be more efficient by reusing existing functions. In Cell 20, the bar width calculation for the cluster bar chart is incorrect, which could lead to overlapping bars. These issues have a strong influence on the conversation flow and usefulness, but they do not render the conversation ineffective.

# Score: 4



In [80]:
print_as_markdown(make_review_request(notebook))

# Critical Severity Issues
# Medium Severity Issues
- Cell 8: The code snippet uses a dictionary with string keys to store algorithms and whether they need scaling. It would be more appropriate to use a named tuple or a simple class to represent each algorithm and its properties for better code clarity and type safety.
- Cell 12: The `run_classifiers` function in `classifiers_set.py` should include a random_state parameter for the train_test_split function to ensure reproducibility of the results.
- Cell 14: The `add_noise` function should check if the noise_level is within a valid range (e.g., 0 to 1) to prevent potential errors or unexpected behavior.
- Cell 16: The `grid_search_noise_levels` function should include a random_state parameter for the train_test_split function within the `run_experiment` function to ensure reproducibility of the results.
- Cell 20: The `plot_results_cluster_bar` function uses a hardcoded threshold (0.001) to determine which noise levels to include in the plot. This threshold should be defined as a constant or parameter to improve code readability and maintainability.
# Scoring Explanation
The code provided by the Assistant is correct and functional, with no critical issues that would make the conversation not useful for the human user. However, there are several medium severity issues related to code clarity, maintainability, and best practices. The use of dictionaries for storing algorithms and their properties could be improved by using named tuples or classes. The absence of a random_state parameter in some functions could lead to non-reproducible results. The hardcoded threshold in the plotting function should be parameterized. Despite these issues, the code achieves the desired functionality and follows good practices in terms of readability and style consistency.

# Score: 4



In [81]:
print_as_markdown(make_review_request(notebook))

# Critical Severity Issues
# Medium Severity Issues
- Cell 8: The code snippet uses a dictionary with string keys 'algo' and 'scale' to store classifiers and whether they need scaling. However, when accessing these values in the loop, the code incorrectly uses algorithm['scale'] and algorithm['algo'] as if 'algorithm' is a dictionary, but it should be algorithm['scale'] and algorithm['algo']. To fix this, replace algorithm['scale'] and algorithm['algo'] with algorithm['scale'] and algorithm['algo'] respectively.
- Cell 12: The code snippet defines a function 'run_classifiers' that accepts a dataset and runs multiple classifiers on it. However, the function does not return any value, which makes it impossible to use the function's results later on. To fix this, add a return statement at the end of the function to return the 'results' list.
- Cell 14: The code snippet provided for 'test_with_noise.py' is missing the import statement for numpy, which is used within the 'add_noise' function. To fix this, add 'import numpy as np' at the beginning of the file.
- Cell 16: The code snippet in 'noise_level_search_on_iris.py' is intended to perform a grid search over noise levels, but it does not return the results from 'run_experiment' function. To fix this, the 'run_experiment' function should return its results, and 'grid_search_noise_levels' should collect these results to be used for plotting or further analysis.
- Cell 18: The code snippet in 'noise_level_search_on_iris.py' is missing the import statement for 'run_experiment' from 'test_with_noise'. To fix this, add 'from test_with_noise import run_experiment' at the beginning of the file.
- Cell 20: The code snippet provided for plotting the cluster bar chart does not correctly calculate the width of the bars based on the number of classifiers. The width calculation 'width = 0.1 / len(classifier_names)' should be adjusted to ensure that the bars are thin and do not overlap. Additionally, the print statements used for debugging should be removed from the final code.
# Scoring Explanation
The code provided by the Assistant is mostly correct and follows best practices, including the use of scikit-learn's API for machine learning tasks and data preprocessing. However, there are several medium severity issues related to incorrect dictionary access, missing return statements, missing import statements, and incorrect bar width calculation for plotting. These issues affect the usability of the code and need to be addressed for the code to function as intended. There are no critical severity issues that would make the conversation not useful for the human user.

# Score: 3



In [82]:
print_as_markdown(make_review_request(notebook))

# Critical Severity Issues
# Medium Severity Issues
- Cell 8: The code snippet uses a dictionary with string keys to store algorithms and whether they need scaling. It would be more Pythonic to use a namedtuple or a simple class to represent each algorithm and its properties, which would improve readability and maintainability.
- Cell 12: The code snippet defines a function `run_classifiers` that returns a list of dictionaries. It would be better to return a list of namedtuples or objects for improved readability and maintainability.
- Cell 14: The function `add_noise` uses `X_train` as a parameter name, but it should be more generic like `X` since it can be applied to any feature set, not just the training set.
- Cell 16: The code snippet uses a hard-coded step size for noise levels (5% increments). It would be more flexible to allow the user to specify the step size as a parameter.
- Cell 18: The code snippet uses `print` statements for debugging purposes (`print('-'*60)`). It would be better to remove or comment out these lines for the final version of the script.
- Cell 20: The function `plot_results_cluster_bar` uses a print statement to output `all_results` and `noise_levels` which seems to be for debugging purposes. It's a good practice to remove or comment out such print statements in the final version of the code.
# Scoring Explanation
The code provided by the Assistant is correct and functional, with no critical issues that would prevent it from being useful to the user. However, there are several medium severity issues related to best practices and maintainability, such as the use of dictionaries where namedtuples or classes would be more appropriate, and the presence of debugging print statements in the final code. These issues do not significantly impact the functionality of the code but could be improved for better code quality. Therefore, the score reflects that the code is generally good but with room for improvement in terms of best practices.

# Score: 4



In [72]:
global_usage_manager.print_costs()

Total cost for all models: $0.000


In [71]:
global_usage_manager.reset_usage()

In [16]:
cells = [{'type': cell['cell_type'], 'content': cell['source']} for cell in notebook['cells']]
cells


[{'type': 'markdown',
  'content': '# Metadata\n\n**Python Topics** - Machine Learning using scikit-learn'},
 {'type': 'markdown', 'content': '# Conversation'},
 {'type': 'markdown',
  'content': "**User**\n\nDo you know the iris dataset for ml? I wanted to explore scikit learn algorithms on it. Give me the code to solve it using some algo from scikit learn. I'm in an empty folder right now. linux"},
 {'type': 'markdown',
  'content': '**Assistant**\n\nYes, I know the iris dataset that is commonly used as a toy dataset to showcase the work of machine learning classification algorithms.\n\nscikit-learn has multiple algorithms that are capable of "solving" it, let\'s start with a simple logistic regression.\n\nBut first, to start we will need to set up an environment with scikit-learn python package installed.\n\nAssuming that you are in a project root directory, let\'s set up a virtual environment for this task.\n\nRun the following in the command line:\n\n```\npython -m venv venv\n```\

In [11]:
global_usage_manager._usage_dict

defaultdict(<function src.llm_reviewer.llm_api.GlobalUsageManager.__init__.<locals>.<lambda>()>,
            {'gpt-4-1106-preview': defaultdict(int,
                         {'completion_tokens': 164,
                          'prompt_tokens': 5207,
                          'total_tokens': 5371})})

In [3]:
from src.llm_reviewer.notebook_reviewer import review_notebook
import nbformat

fn = 'src/llm_reviewer/tests/samples/10T__ml_research.ipynb'
notebook = nbformat.read(fn, as_version=4)

per_turn_review = review_notebook(
    {'nb_parsed_notebook': notebook, 'file_id': fn},
    max_threads=10,
    progress_counter= None,
    verbose = 2,
)

/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Reviews done: 0, Reviews left after this one: 19
Reviews done: 0, Reviews left after this one: 18
Reviews done: 0, Reviews left after this one: 17
Reviews done: 0, Reviews left after this one: 16
Reviews done: 0, Reviews left after this one: 15
Reviews done: 0, Reviews left after this one: 14
Reviews done: 0, Reviews left after this one: 13
Reviews done: 0, Reviews left after this one: 12
Reviews done: 0, Reviews left after this one: 11
Reviews done: 0, Reviews left after this one: 10
Review for turn_id=1 by reviewer='code_reviewer' is done. 1 / 20 reviews completed.
Reviews done: 1, Reviews left after this one: 9
Review for turn_id=3 by reviewer='code_reviewer' is done. 2 / 20 reviews completed.
Reviews done: 2, Reviews left after this one: 8
Review for turn_id=4 by reviewer='english_reviewer' is done. 3 / 20 reviews completed.
Reviews done: 3, Reviews left after this one: 7
Review for turn_id=2 by reviewer='english_reviewer' is done. 4 / 20 reviews completed.
Reviews done: 4, Reviews

In [6]:
from src.llm_reviewer.notebook_reviewer import notebook_reviews_to_df, IssueLevel

df = notebook_reviews_to_df(filter(None, [per_turn_review]), IssueLevel.MEDIUM)
get_judgment('code', df.iloc[0]['code_feedback'])

{'judgment': 'The code reviews consistently identified critical issues such as missing exception handling, lack of main guards, mismatched function definitions, and missing imports, alongside medium issues related to code efficiency and robustness.',
 'score': '3'}